<a href="https://colab.research.google.com/github/bogdanovap/DSNN-5/blob/main/%D0%9F%D1%80%D0%BE%D0%B3%D0%BD%D0%BE%D0%B7%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BA%D0%B0%D1%87%D0%B5%D1%81%D1%82%D0%B2%D0%B0_%D0%B2%D0%B8%D0%BD%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Понимание бизнеса

## 1.1 Цель
Предсказать качество вина

## 1.2 Описание
Data Set Information:

The dataset was downloaded from the UCI Machine Learning Repository.

The two datasets are related to red and white variants of the Portuguese "Vinho Verde" wine. The reference [Cortez et al., 2009]. Due to privacy and logistic issues, only physicochemical (inputs) and sensory (the output) variables are available (e.g. there is no data about grape types, wine brand, wine selling price, etc.).

These datasets can be viewed as classification or regression tasks. The classes are ordered and not balanced (e.g. there are munch more normal wines than excellent or poor ones). Outlier detection algorithms could be used to detect the few excellent or poor wines. Also, we are not sure if all input variables are relevant. So it could be interesting to test feature selection methods.

Two datasets were combined and few values were randomly removed.

[Описание от Kaggle](https://www.kaggle.com/datasets/rajyellow46/wine-quality)

# 2. Data Understanding

## 2.1 Import Libraries

In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Handle table-like data and matrices
import numpy as np
import pandas as pd

# Modelling Algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier

# Modelling Helpers
from sklearn.impute import SimpleImputer as Imputer
from sklearn.preprocessing import  Normalizer , scale
from sklearn.model_selection import train_test_split , StratifiedKFold
from sklearn.feature_selection import RFECV

# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# Configure visualisations
%matplotlib inline
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 8 , 6

## 2.2 Вспомогательные функции

In [ ]:
def plot_histograms( df , variables , n_rows , n_cols ):
    fig = plt.figure( figsize = ( 16 , 12 ) )
    for i, var_name in enumerate( variables ):
        ax=fig.add_subplot( n_rows , n_cols , i+1 )
        df[ var_name ].hist( bins=10 , ax=ax )
        ax.set_title( 'Skew: ' + str( round( float( df[ var_name ].skew() ) , ) ) ) # + ' ' + var_name ) #var_name+" Distribution")
        ax.set_xticklabels( [] , visible=False )
        ax.set_yticklabels( [] , visible=False )
    fig.tight_layout()  # Improves appearance a bit.
    plt.show()

def plot_distribution( df , var , target , **kwargs ):
    row = kwargs.get( 'row' , None )
    col = kwargs.get( 'col' , None )
    facet = sns.FacetGrid( df , hue=target , aspect=4 , row = row , col = col )
    facet.map( sns.kdeplot , var , shade= True )
    facet.set( xlim=( 0 , df[ var ].max() ) )
    facet.add_legend()

def plot_categories( df , cat , target , **kwargs ):
    row = kwargs.get( 'row' , None )
    col = kwargs.get( 'col' , None )
    facet = sns.FacetGrid( df , row = row , col = col )
    facet.map( sns.barplot , cat , target )
    facet.add_legend()

def plot_correlation_map( df ):
    corr = df.corr()
    _ , ax = plt.subplots( figsize =( 12 , 10 ) )
    cmap = sns.diverging_palette( 220 , 10 , as_cmap = True )
    _ = sns.heatmap(
        corr, 
        cmap = cmap,
        square=True, 
        cbar_kws={ 'shrink' : .9 }, 
        ax=ax, 
        annot = True, 
        annot_kws = { 'fontsize' : 12 }
    )

def describe_more( df ):
    var = [] ; l = [] ; t = []
    for x in df:
        var.append( x )
        l.append( len( pd.value_counts( df[ x ] ) ) )
        t.append( df[ x ].dtypes )
    levels = pd.DataFrame( { 'Variable' : var , 'Levels' : l , 'Datatype' : t } )
    levels.sort_values( by = 'Levels' , inplace = True )
    return levels

def plot_variable_importance( X , y ):
    tree = DecisionTreeClassifier( random_state = 99 )
    tree.fit( X , y )
    plot_model_var_imp( tree , X , y )
    
def plot_model_var_imp( model , X , y ):
    imp = pd.DataFrame( 
        model.feature_importances_  , 
        columns = [ 'Importance' ] , 
        index = X.columns 
    )
    imp = imp.sort_values( [ 'Importance' ] , ascending = True )
    imp[ : 10 ].plot( kind = 'barh' )
    print (model.score( X , y ))

## 2.3 Загрузка данных

In [ ]:
wines = pd.read_csv("winequalityN.csv")

print ('Datasets:' , 'wines:' , wines.shape)

FileNotFoundError: ignored

## 2.4 Статистика и визуализации

In [ ]:
wines.head()

**Описание переменных**

For more information, read [Cortez et al., 2009].
Input variables (based on physicochemical tests):
- fixed acidity
- volatile acidity
- citric acid
- residual sugar
- chlorides
- free sulfur dioxide
- total sulfur dioxide
- density
- pH
- sulphates
- alcohol

Output variable (based on sensory data):
- quality (score between 0 and 10)

[Больше информации на сайте Kaggle](https://www.kaggle.com/datasets/rajyellow46/wine-quality)

### 2.4.1 Далее взглянем на некоторую ключевую информацию о переменных
Числовая переменная - это переменная со значениями в области целых или действительных чисел, в то время как категориальная переменная - это переменная, которая может принимать одно из ограниченного и обычно фиксированного числа возможных значений, таких как тип крови.

Обратите особое внимание на тип каждой переменной, количество наблюдений и некоторые значения переменных.

Интересным наблюдением может быть, например, минимальный возраст 0,42.

Выводы:
- не хватает знаний для оценки допустимости значений показателей. Будем считать что данные, в части значений, качественные
- есть пропуски данных почти по всем показателям. необходимо удалить строки с пропусками

In [ ]:
wines.describe()

### 2.4.2 Тепловая карта корреляции может дать нам понимание того, какие переменные важны

In [ ]:
plot_correlation_map( wines )

Выводы:
- нет показателей, которые можно назвать "лишними", т.к. каждый показаатель имеет значимую корелляцию с одним и более показателем
- с другой стороны, есть показатели, которые оказывают слабое влияние на качество вина: fixed acidity, citric acid, residual sugar, free sulfur dioxide, total sulfur dioxide, pH, sulphates

### 2.4.3 Давайте подробнее рассмотрим взаимосвязь между признаками и качество вина
рассмотрим пример с высокой корреляцией и низкой

In [ ]:
plot_distribution( wines , var = 'volatile acidity' , target = 'quality' , row = 'type' )

In [ ]:
plot_distribution( wines , var = 'pH' , target = 'quality' , row = 'type' )

вывод:
- мы не можем отвергнуть гипотезу о том, что факторы с низкой корреляцией не значимы для качества вина. необходимо проверять эмпирически при обучении модели

### 2.4.4 Тип вина
Мы также можем посмотреть на категориальные переменные

In [ ]:
plot_categories( wines , cat = 'type' , target = 'quality' )

# 3. Data Preparation

## 3.1 Категориальные переменные должны быть преобразованы в числовые переменные

In [ ]:
# Transform Sex into binary values 0 and 1
wine_type = pd.Series( np.where( wines.type == 'white' , 1 , 0 ) , name = 'type' )

## 3.2 Заполнить пропущенные значения в переменных
Большинство алгоритмов машинного обучения требуют, чтобы все переменные имели значения, чтобы использовать их для обучения модели. Самый простой метод - заполнить пропущенные значения средним по переменной для всех наблюдений в обучающем наборе.

In [ ]:
# Create dataset
imputed = pd.DataFrame()

numeric_columns = [ "fixed acidity",
                    "volatile acidity",
                    "citric acid",
                    "residual sugar",
                    "chlorides",
                    "free sulfur dioxide",
                    "total sulfur dioxide",
                    "density",
                    "pH",
                    "sulphates",
                    "alcohol"
                  ]
wines[numeric_columns] = wines[numeric_columns].fillna(wines[numeric_columns].mean())

wines.describe()

## 3.4 Сборка финальных датасетов для моделирования

### 3.4.1 Variable selection

In [ ]:
full_X = pd.concat( [ wine_type, wines[numeric_columns] ] , axis=1 )
full_X.head()

### 3.4.2 Создание датасетов

Отделяем данные для обучения и для проверки

In [ ]:
train_X , valid_X , train_y , valid_y = train_test_split( full_X , wines.quality , train_size = .7 )

### 3.4.3 Важность признаков
Отбор оптимальных признаков для модели имеет важное значение. Теперь мы попытаемся оценить, какие переменные являются наиболее важными, чтобы сделать прогноз.

In [ ]:
plot_variable_importance(train_X, train_y)

# 4. Моделирование
Теперь мы выберем модель, которую хотели бы попробовать. Используем обучающий набор данных для обучения модели и затем проверим ее с помощью тестового набора.

## 4.1 Выбор модели
Хорошей отправной точкой является регрессия.

In [ ]:
model_1 = GradientBoostingClassifier()
model_2 = RandomForestClassifier()

## 4.2 Обучение модели

In [ ]:
model_1.fit( train_X , train_y )
model_2.fit( train_X , train_y )

# 5. Оценка
Теперь мы собираемся оценить модель

## 5.1 Модель
Мы можем оценить точность модели, используя набор для валидации, где мы знаем фактический результат. Этот набор данных не использовался для обучения, поэтому он абсолютно новый для модели.

Затем мы сравниваем точность с точностью при использовании модели на тренировочных данных. Если разница между ними значительна, это свидетельствует о переобучении. Мы стараемся избегать этого, потому что это означает, что модель не будет хорошо обобщаться на новые данные (будет работать плохо)

In [ ]:
# Score the model
print (model_1.score( train_X , train_y ) , model_1.score( valid_X , valid_y ))

In [ ]:

print (model_2.score( train_X , train_y ) , model_2.score( valid_X , valid_y ))